# Shape → Road Link Mapping (Basics)
This notebook focuses on a minimal, inspectable mapping from a single GTFS shape_id to OSM road links.

What it does:
- Select candidate road links near the shape
- Compute overlap length with a small buffer around the shape
- Apply absolute and ratio thresholds to reduce false positives
- Preview results (counts + sample)
- Optional: materialize to a clean table one shape at a time

In [5]:
# Configure variables
import os
from pathlib import Path
from google.cloud import bigquery
from google.oauth2 import service_account
from IPython.display import display

project_id = os.environ.get("BQ_PROJECT", "regal-dynamo-470908-v9")
dataset_id = os.environ.get("BQ_DATASET", "auckland_data_dev")
location = os.environ.get("BQ_LOCATION", "australia-southeast1")
# Set a target shape_id to test mapping (edit this)
shape_id = os.environ.get("SINGLE_SHAPE_ID", "240-96608-0ba0b9ba")

print({"project_id": project_id, "dataset_id": dataset_id, "location": location, "shape_id": shape_id})

{'project_id': 'regal-dynamo-470908-v9', 'dataset_id': 'auckland_data_dev', 'location': 'australia-southeast1', 'shape_id': '240-96608-0ba0b9ba'}


In [6]:
# BigQuery client + helper
credentials_path = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS_JSON")
if credentials_path and Path(credentials_path).exists():
    creds = service_account.Credentials.from_service_account_file(credentials_path)
    client = bigquery.Client(project=project_id, credentials=creds)
else:
    client = bigquery.Client(project=project_id)

def run_sql_loc(sql: str, params=None):
    job_config = bigquery.QueryJobConfig(query_parameters=params or [])
    job = client.query(sql, job_config=job_config, location=location)
    res = job.result()
    print(f"Job {job.job_id} done. bytes={job.total_bytes_processed:,} cache={job.cache_hit}")
    return res.to_dataframe()

In [12]:
# Map a single shape_id to nearby road links
# Tunables (meters):
BUF_M = 6.0        # buffer around the shape for overlap measurement
BBOX_M = 25.0      # bounding box margin around the shape to prefilter roads
OVERLAP_MIN = 10.0 # keep edges with at least this many meters overlapped
RATIO_MIN = 0.15   # keep edges where overlap/edge_length >= this ratio

sql = f"""
WITH s AS (
  SELECT
    shape_id,
    geom,
    ST_BUFFER(geom, @buf_m) AS buf_geom,
    ST_BOUNDINGBOX(ST_BUFFER(geom, @bbox_m)) AS env_box
  FROM `{project_id}.{dataset_id}.shapes_geog`
  WHERE shape_id = @shape_id
), r AS (
  SELECT
    edge_id, name AS road_name, highway, oneway, maxspeed,
    length_m AS edge_length_m, geom AS edge_geom
  FROM `{project_id}.{dataset_id}.vw_osm_akl_road_links`
  WHERE highway NOT IN ('footway','path','cycleway','steps','track','pedestrian','bridleway','corridor')
)
SELECT
  s.shape_id, r.edge_id, r.road_name, r.highway, r.oneway, r.maxspeed, r.edge_length_m,
  ST_LENGTH(ST_INTERSECTION(r.edge_geom, s.buf_geom)) AS overlap_m,
  SAFE_DIVIDE(ST_LENGTH(ST_INTERSECTION(r.edge_geom, s.buf_geom)), r.edge_length_m) AS overlap_ratio,
  r.edge_geom
FROM s
JOIN r
  ON ST_INTERSECTSBOX(r.edge_geom, s.env_box.xmin, s.env_box.ymin, s.env_box.xmax, s.env_box.ymax)
WHERE ST_LENGTH(ST_INTERSECTION(r.edge_geom, s.buf_geom)) > @overlap_min
  AND SAFE_DIVIDE(ST_LENGTH(ST_INTERSECTION(r.edge_geom, s.buf_geom)), r.edge_length_m) >= @ratio_min
"""
params = [
  bigquery.ScalarQueryParameter('shape_id', 'STRING', shape_id),
  bigquery.ScalarQueryParameter('buf_m', 'FLOAT64', BUF_M),
  bigquery.ScalarQueryParameter('bbox_m', 'FLOAT64', BBOX_M),
  bigquery.ScalarQueryParameter('overlap_min', 'FLOAT64', OVERLAP_MIN),
  bigquery.ScalarQueryParameter('ratio_min', 'FLOAT64', RATIO_MIN),
]
df = run_sql_loc(sql, params)
print(f"rows={len(df)}")
display(df.head(20))

Job 1de3051e-0465-4e00-b011-03bb194b30c9 done. bytes=35,073,952 cache=False
rows=177


,shape_id,edge_id,road_name,highway,oneway,maxspeed,edge_length_m,overlap_m,overlap_ratio,edge_geom
0,240-96608-0ba0b9ba,149318811,Onewa Road,primary,None,50,149.450055,149.450055,1.000000,"LINESTRING(174.7461579 -36.8118231, 174.746739..."
1,240-96608-0ba0b9ba,613882668,Onewa Road,primary,None,50,25.030688,25.030688,1.000000,"LINESTRING(174.7428232 -36.8108144, 174.743086..."
2,240-96608-0ba0b9ba,617488818,Khyber Pass Road,primary,None,50,28.910289,28.910289,1.000000,"LINESTRING(174.7705124 -36.8660219, 174.770827..."
3,240-96608-0ba0b9ba,628160158,Jervois Road,secondary,None,50,89.528786,89.528786,1.000000,"LINESTRING(174.7413495 -36.8456053, 174.741892..."
4,240-96608-0ba0b9ba,627274269,Khyber Pass Road,secondary,None,50,72.574964,72.574964,1.000000,"LINESTRING(174.7754956 -36.8670936, 174.776284..."
5,240-96608-0ba0b9ba,158619823,None,motorway_link,yes,50,28.539283,28.539283,1.000000,"LINESTRING(174.7515171 -36.8135865, 174.751495..."
6,240-96608-0ba0b9ba,8371382,Karangahape Road,secondary,None,30,65.846725,65.846725,1.000000,"LINESTRING(174.7556931 -36.8583427, 174.756382..."
7,240-96608-0ba0b9ba,616959767,Birkenhead Avenue,secondary,None,50,14.739428,14.739428,1.000000,"LINESTRING(174.7257859 -36.8108446, 174.725837..."
8,240-96608-0ba0b9ba,7638428,Curran Street,secondary,None,50,399.746183,399.746183,1.000000,"LINESTRING(174.7395739 -36.8410722, 174.739668..."
9,240-96608-0ba0b9ba,813754043,None,service,yes,None,116.419523,28.916076,0.248378,"LINESTRING(174.739711 -36.8382883, 174.7399031..."


## Plot edge geometries (Plotly Express)
Simple map of the road link geometries already in `df` using Plotly Express (no extra queries).

In [13]:
# Plot edge polylines on OpenStreetMap using px.line_mapbox
import json
import pandas as pd
import plotly.express as px
from IPython.display import HTML, display
try:
    from shapely import wkt as shapely_wkt
    from shapely.geometry import LineString, MultiLineString
except Exception:
    shapely_wkt = None
    LineString = MultiLineString = None

if df is None or len(df) == 0:
    print("No edges to plot in df.")
else:
    recs = []
    for _, row in df.iterrows():
        geom = row.get('edge_geom')
        eid = str(row.get('edge_id'))
        if geom is None:
            continue
        coords_groups = []
        # GeoJSON string path
        gj = None
        if isinstance(geom, str):
            try:
                gj = json.loads(geom)
            except Exception:
                gj = None
        if gj and isinstance(gj, dict):
            typ = gj.get('type')
            if typ == 'LineString':
                coords_groups = [gj['coordinates']]
            elif typ == 'MultiLineString':
                coords_groups = gj['coordinates']
        # __geo_interface__ path
        if not coords_groups and hasattr(geom, '__geo_interface__'):
            gi = geom.__geo_interface__
            typ = gi.get('type')
            if typ == 'LineString':
                coords_groups = [gi['coordinates']]
            elif typ == 'MultiLineString':
                coords_groups = gi['coordinates']
        # WKT path via shapely if available
        if not coords_groups and isinstance(geom, str) and shapely_wkt is not None:
            try:
                g = shapely_wkt.loads(geom)
                if isinstance(g, LineString):
                    coords_groups = [list(g.coords)]
                elif isinstance(g, MultiLineString):
                    coords_groups = [list(ls.coords) for ls in g.geoms]
            except Exception:
                pass
        for idx, coords in enumerate(coords_groups):
            for lon, lat in coords:
                recs.append({'lon': float(lon), 'lat': float(lat), 'segment': f"{eid}-{idx}", 'edge_id': eid})
    if not recs:
        print("Could not parse geometries from df.edge_geom.")
    else:
        gdf = pd.DataFrame(recs)
        center_lat = float(gdf['lat'].mean())
        center_lon = float(gdf['lon'].mean())
        fig = px.line_mapbox(
            gdf, lon='lon', lat='lat', line_group='segment', color='edge_id', height=560, zoom=13
        )
        # Use a paler basemap (no token required)
        fig.update_layout(mapbox_style='carto-positron', mapbox_center={'lat': center_lat, 'lon': center_lon}, margin=dict(l=0,r=0,t=0,b=0))
        try:
            fig.show()
        except Exception as e:
            html = fig.to_html(include_plotlyjs='cdn', full_html=False)
            display(HTML(html))
            print(f"Rendered via HTML fallback (renderer error: {e})")

/var/folders/yb/hshltv157x9fyt_dsl7vnn940000gn/T/ipykernel_10505/940428077.py:63: DeprecationWarning:

*line_mapbox* is deprecated! Use *line_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



## Diagnostic: unmatched nearby OSM links
Check which OSM road links near the shape were not selected into `df` and how much they overlap; helps explain gaps (e.g., short connectors filtered by thresholds).

In [5]:
# Diagnostic query: nearby OSM links not in df (same highway filter)
from google.cloud import bigquery
import pandas as pd

if df is None or len(df) == 0:
    print("df is empty; run the mapping cell above first.")
else:
    matched_ids = set(df['edge_id'].dropna().astype(str))
    diag_sql = f"""
    WITH s AS (
      SELECT
        shape_id,
        ST_BUFFER(geom, @buf_m) AS buf_geom,
        ST_BOUNDINGBOX(ST_BUFFER(geom, @bbox_m)) AS env_box
      FROM `{project_id}.{dataset_id}.shapes_geog`
      WHERE shape_id = @shape_id
    ), r AS (
      SELECT
        edge_id, name AS road_name, highway, oneway, maxspeed,
        length_m AS edge_length_m, geom AS edge_geom
      FROM `{project_id}.{dataset_id}.vw_osm_akl_road_links`
      WHERE highway NOT IN ('footway','path','cycleway','steps','track','pedestrian','bridleway','corridor')
    )
    SELECT
      r.edge_id, r.road_name, r.highway, r.oneway, r.maxspeed, r.edge_length_m,
      ST_LENGTH(ST_INTERSECTION(r.edge_geom, s.buf_geom)) AS overlap_m,
      SAFE_DIVIDE(ST_LENGTH(ST_INTERSECTION(r.edge_geom, s.buf_geom)), r.edge_length_m) AS overlap_ratio
    FROM s
    JOIN r
      ON ST_INTERSECTSBOX(r.edge_geom, s.env_box.xmin, s.env_box.ymin, s.env_box.xmax, s.env_box.ymax)
    """
    dparams = [
        bigquery.ScalarQueryParameter('shape_id', 'STRING', shape_id),
        bigquery.ScalarQueryParameter('buf_m', 'FLOAT64', BUF_M),
        bigquery.ScalarQueryParameter('bbox_m', 'FLOAT64', BBOX_M),
    ]
    candidates = run_sql_loc(diag_sql, dparams)
    print(f"candidates={len(candidates)}")
    candidates['edge_id_str'] = candidates['edge_id'].astype(str)
    unmatched = candidates[~candidates['edge_id_str'].isin(matched_ids)].copy()
    print(f"matched_unique={len(matched_ids)}  unmatched={len(unmatched)}")
    if len(unmatched) == 0:
        print("All nearby candidate links are already in df — gaps likely reflect OSM geometry (e.g., split ways or true gaps).")
    else:
        # Overlap stats for unmatched
        unmatched['overlap_m'] = unmatched['overlap_m'].astype(float)
        unmatched['overlap_ratio'] = unmatched['overlap_ratio'].astype(float)
        pos = unmatched[unmatched['overlap_m'] > 0].sort_values('overlap_m', ascending=False)
        print(f"unmatched_with_overlap>0 = {len(pos)}")
        # Borderline near thresholds (half of current thresholds)
        borderline = pos[(pos['overlap_m'] >= 0.5*OVERLAP_MIN) | (pos['overlap_ratio'] >= 0.5*RATIO_MIN)]
        print(f"borderline_near_thresholds = {len(borderline)} (half-threshold check)")
        # Show a sample
        display(pos[['edge_id','road_name','highway','edge_length_m','overlap_m','overlap_ratio']].head(20))
        # Quick aggregation to see if filtered types dominate
        by_hw = pos.groupby('highway', dropna=False).agg(count=('edge_id','count'),
                                                         sum_overlap=('overlap_m','sum'),
                                                         median_overlap=('overlap_m','median')).sort_values('sum_overlap', ascending=False)
        display(by_hw.head(10))
        
        print("Notes:")
        print("- If many unmatched have meaningful overlap, our thresholds (OVERLAP_MIN/RATIO_MIN) may be too strict.")
        print("- If unmatched are mostly service/turning_slip/etc., gaps may be expected connectors.")
        print("- If no unmatched-with-overlap, gaps follow OSM geometry (split ways or true gaps)." )

Job 1285329f-c9b8-4c14-af6a-3a550b27f665 done. bytes=35,073,952 cache=False
candidates=5368
matched_unique=89  unmatched=5279
unmatched_with_overlap>0 = 292
borderline_near_thresholds = 213 (half-threshold check)
candidates=5368
matched_unique=89  unmatched=5279
unmatched_with_overlap>0 = 292
borderline_near_thresholds = 213 (half-threshold check)


,edge_id,road_name,highway,edge_length_m,overlap_m,overlap_ratio
5274,616787771,Broadway,secondary,39.499514,39.499514,1.00000
3959,330604958,Auckland Northern Motorway,motorway,38.633379,38.633379,1.00000
5125,822845477,Karangahape Road,secondary,38.570633,38.570633,1.00000
3044,276984117,Karangahape Road,secondary,38.301269,38.301269,1.00000
4422,623580294,Grafton Bridge,secondary,37.539924,37.539924,1.00000
3975,614967567,Onewa Road,primary,37.447442,37.447442,1.00000
3593,612703680,Ponsonby Road,secondary,37.288679,37.288679,1.00000
1731,805519475,Ponsonby Road,secondary,37.002426,37.002426,1.00000
4264,613882670,Onewa Road,primary,36.464393,36.464393,1.00000
3875,612703681,Ponsonby Road,secondary,36.306842,36.306842,1.00000


,count,sum_overlap,median_overlap
highway,,,
secondary,80,1460.377306,14.828919
service,80,535.834565,5.782428
primary,20,404.368865,20.899524
residential,39,274.614271,6.166688
motorway_link,17,226.736632,12.021482
unclassified,25,166.644914,5.525672
motorway,9,155.698277,12.009949
secondary_link,6,78.203425,11.145399
tertiary,11,71.495701,5.249681


Notes:
- If many unmatched have meaningful overlap, our thresholds (OVERLAP_MIN/RATIO_MIN) may be too strict.
- If unmatched are mostly service/turning_slip/etc., gaps may be expected connectors.
- If no unmatched-with-overlap, gaps follow OSM geometry (split ways or true gaps).


## Lookup OSM links by road name
Query all links from `osm_akl_road_links` for a given `name` (with a safe fallback to the view if needed).

In [8]:
# Query OSM links by exact road name
from google.cloud import bigquery
from IPython.display import display
import os

# Set the road name to filter by (edit as needed)
road_name = os.environ.get("OSM_ROAD_NAME", "Symonds Street")
print({"road_name": road_name})

# Prefer the base table; if it doesn't exist in this dataset, fall back to the view
sql_table = f"""
SELECT
  edge_id,
  name AS road_name,
  highway,
  oneway,
  maxspeed,
  length_m AS edge_length_m,
  geom AS edge_geom
FROM `{project_id}.{dataset_id}.osm_akl_road_links`
WHERE name = @road_name
ORDER BY edge_id
"""

sql_view_fallback = f"""
SELECT
  edge_id,
  name AS road_name,
  highway,
  oneway,
  maxspeed,
  length_m AS edge_length_m,
  geom AS edge_geom
FROM `{project_id}.{dataset_id}.vw_osm_akl_road_links`
WHERE name = @road_name
ORDER BY edge_id
"""

params = [bigquery.ScalarQueryParameter("road_name", "STRING", road_name)]

try:
  df_by_name = run_sql_loc(sql_table, params)
except Exception as e:
  print(f"Base table query failed, trying view fallback: {e}")
  df_by_name = run_sql_loc(sql_view_fallback, params)

print(f"rows={len(df_by_name)}")
display(df_by_name.head(50))

{'road_name': 'Symonds Street'}
Job 96ad0f7d-00ef-47bd-8bd9-7a8428882cee done. bytes=16,063,032 cache=False
Job 96ad0f7d-00ef-47bd-8bd9-7a8428882cee done. bytes=16,063,032 cache=False
Job 96ad0f7d-00ef-47bd-8bd9-7a8428882cee done. bytes=16,063,032 cache=False
rows=55
rows=55
rows=55


,edge_id,road_name,highway,oneway,maxspeed,edge_length_m,edge_geom
0,7637280,Symonds Street,unclassified,None,50,60.840531,"LINESTRING(174.7758093 -36.912713, 174.7758584..."
1,7637281,Symonds Street,residential,None,50,579.494817,"LINESTRING(174.7733312 -36.9217472, 174.773319..."
2,24457402,Symonds Street,secondary,None,50,134.811280,"LINESTRING(174.7627793 -36.8598013, 174.763089..."
3,24457404,Symonds Street,secondary,None,50,87.369666,"LINESTRING(174.7620581 -36.8610318, 174.762087..."
4,24457649,Symonds Street,primary,None,50,11.054568,"LINESTRING(174.760914 -36.86404, 174.760906 -3..."
5,32799524,Symonds Street,secondary,None,30,7.132385,"LINESTRING(174.7685063 -36.8537484, 174.768556..."
6,37559107,Symonds Street,primary,yes,50,119.712804,"LINESTRING(174.761985 -36.8611502, 174.7619806..."
7,37559126,Symonds Street,primary,yes,50,206.618088,"LINESTRING(174.7609183 -36.863737, 174.7609351..."
8,52728801,Symonds Street,secondary,None,30,112.151836,"LINESTRING(174.7647254 -36.85732, 174.7648638 ..."
9,52728937,Symonds Street,secondary,None,30,200.754464,"LINESTRING(174.7705009 -36.8517601, 174.770589..."


## Plot OSM links (queried by name) on a map
Visualize the `edge_geom` from `df_by_name` using Plotly Express on a carto-positron basemap.

In [11]:
# Plot df_by_name edge polylines on a paler basemap
import json
import pandas as pd
import plotly.express as px
from IPython.display import HTML, display
try:
    from shapely import wkt as shapely_wkt
    from shapely.geometry import LineString, MultiLineString
except Exception:
    shapely_wkt = None
    LineString = MultiLineString = None

if 'df_by_name' not in globals() or df_by_name is None or len(df_by_name) == 0:
    print("No rows in df_by_name to plot. Run the query cell above first.")
else:
    recs = []
    for _, row in df_by_name.iterrows():
        geom = row.get('edge_geom')
        eid = str(row.get('edge_id'))
        if geom is None:
            continue
        coords_groups = []
        # Try GeoJSON string
        gj = None
        if isinstance(geom, str):
            try:
                gj = json.loads(geom)
            except Exception:
                gj = None
        if gj and isinstance(gj, dict):
            typ = gj.get('type')
            if typ == 'LineString':
                coords_groups = [gj['coordinates']]
            elif typ == 'MultiLineString':
                coords_groups = gj['coordinates']
        # __geo_interface__
        if not coords_groups and hasattr(geom, '__geo_interface__'):
            gi = geom.__geo_interface__
            typ = gi.get('type')
            if typ == 'LineString':
                coords_groups = [gi['coordinates']]
            elif typ == 'MultiLineString':
                coords_groups = gi['coordinates']
        # WKT via shapely if available
        if not coords_groups and isinstance(geom, str) and shapely_wkt is not None:
            try:
                g = shapely_wkt.loads(geom)
                if isinstance(g, LineString):
                    coords_groups = [list(g.coords)]
                elif isinstance(g, MultiLineString):
                    coords_groups = [list(ls.coords) for ls in g.geoms]
            except Exception:
                pass
        for idx, coords in enumerate(coords_groups):
            for lon, lat in coords:
                recs.append({'lon': float(lon), 'lat': float(lat), 'segment': f"{eid}-{idx}", 'edge_id': eid,
                             'road_name': row.get('road_name'), 'highway': row.get('highway')})
    if not recs:
        print("Could not parse geometries from df_by_name.edge_geom.")
    else:
        gdf = pd.DataFrame(recs)
        center_lat = float(gdf['lat'].mean())
        center_lon = float(gdf['lon'].mean())
        fig = px.line_mapbox(
            gdf,
            lon='lon', lat='lat', line_group='segment', color='edge_id',
            hover_data={'edge_id': True, 'road_name': True, 'highway': True, 'lon': False, 'lat': False},
            height=560, zoom=13,
        )
        fig.update_layout(
            mapbox_style='carto-positron',
            mapbox_center={'lat': center_lat, 'lon': center_lon},
            margin=dict(l=0, r=0, t=0, b=0)
        )
        try:
            fig.show()
        except Exception as e:
            html = fig.to_html(include_plotlyjs='cdn', full_html=False)
            display(HTML(html))
            print(f"Rendered via HTML fallback (renderer error: {e})")

/var/folders/yb/hshltv157x9fyt_dsl7vnn940000gn/T/ipykernel_10505/3291254850.py:64: DeprecationWarning:

*line_mapbox* is deprecated! Use *line_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

